<a href="https://colab.research.google.com/github/faiez22222/Credit-Card-Validator/blob/main/summarizationtechnique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip install sentencepiece

In [4]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [6]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by
              triple backquotes.
              Return your response in bullet points which covers the key
              points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """ As part of Meta’s commitment to open science, today we are publicly
releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational
large language model designed to help researchers advance their work in this
subfield of AI. Smaller, more performant models such as LLaMA enable others
in the research community who don’t have access to large amounts of
infrastructure to study these models, further democratizing access in this
important, fast-changing field.Training smaller foundation models like LLaMA
is desirable in the large language model space because it requires far less
computing power and resources to test new approaches, validate others’ work,
and explore new use cases. Foundation models train on a large set of unlabeled
data, which makes them ideal for fine-tuning for a variety of tasks.
We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B
parameters) and also sharing a LLaMA model card that details how we built
the model in keeping with our approach to Responsible AI practices.
Over the last year, large language models — natural language processing (NLP)
systems with billions of parameters — have shown new capabilities to generate
creative text, solve mathematical theorems, predict protein structures,
answer reading comprehension questions, and more. They are one of the clearest
cases of the substantial potential benefits AI can offer at scale to billions
of people.

Even with all the recent advancements in large language models, full research
access to them remains limited because of the resources that are required to
train and run such large models. This restricted access has limited researchers
’ ability to understand how and why these large language models work,
hindering progress on efforts to improve their robustness and mitigate known
issues, such as bias, toxicity, and the potential for generating misinformation.

Smaller models trained on more tokens — which are pieces of words — are easier
to retrain and fine-tune for specific potential product use cases.
We trained LLaMA 65B and LLaMA 33B on 1.4 trillion tokens. Our smallest model,
LLaMA 7B, is trained on one trillion tokens.

Like other large language models, LLaMA works by taking a sequence of words
as an input and predicts a next word to recursively generate text.
To train our model, we chose text from the 20 languages with the most speakers,
focusing on those with Latin and Cyrillic alphabets.

There is still more research that needs to be done to address the risks of
bias, toxic comments, and hallucinations in large language models.
Like other models, LLaMA shares these challenges. As a foundation model, LLaMA
is designed to be versatile and can be applied to many different use cases,
versus a fine-tuned model that is designed for a specific task.
By sharing the code for LLaMA, other researchers can more easily test new
approaches to limiting or eliminating these problems in large language models.
We also provide in the paper a set of evaluations on benchmarks evaluating
model biases and toxicity to show the model’s limitations and to support
further research in this crucial area.

To maintain integrity and prevent misuse, we are releasing our model under
a noncommercial license focused on research use cases. Access to the model
will be granted on a case-by-case basis to academic researchers; those
affiliated with organizations in government, civil society, and academia;
and industry research laboratories around the world. People interested in
applying for access can find the link to the application in our research paper.

We believe that the entire AI community — academic researchers, civil society,
policymakers, and industry — must work together to develop clear guidelines
around responsible AI in general and responsible large language models
in particular. We look forward to seeing what the community can learn —
and eventually build — using LLaMA.
"""
print(llm_chain.run(text))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 • Meta is releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational large language model designed to advance research in this subfield of AI.
            • Training smaller foundation models like LLaMA is desirable in the large language model space because it requires less computing power and resources.
            • LLaMA is available at several sizes (7B, 13B, 33B, and 65B parameters) and a model card details how the model was built.
            • Large language models have shown new capabilities, but full research access remains limited due to resource requirements.
            • Smaller models are easier to retrain and fine-tune for specific product use cases.
            • LLaMA works by taking a sequence of words as an input and predicting a next word to generate text.
            • The model was trained on 1.4 trillion tokens, with the smallest model, LLaMA 7B, trained on one trillion tokens.
            • There is still more research needed to address ris

In [7]:
text1 = """
Tesla, Inc. (/ˈtɛslə/ TESS-lə or /ˈtɛzlə/ TEZ-lə[a]) is an American
multinational automotive and clean energy company headquartered in Austin,
Texas. Tesla designs and manufactures electric vehicles (cars and trucks),
stationary battery energy storage devices from home to grid-scale,
solar panels and solar roof tiles, and related products and services.

Tesla is one of the world's most valuable companies and, as of 2023,
was the world's most valuable automaker. In 2022, the company led the
battery electric vehicle market, with 18% share.

Its subsidiary Tesla Energy develops and is a major installer of photovoltaic
systems in the United States. Tesla Energy is one of the largest global
suppliers of battery energy storage systems, with 6.5 gigawatt-hours (GWh)
installed in 2022.

Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as
Tesla Motors. The company's name is a tribute to inventor and
electrical engineer Nikola Tesla. In February 2004, via a $6.5 million
investment, Elon Musk became the company's largest shareholder.
He became CEO in 2008. Tesla's announced mission is to help expedite
the move to sustainable transport and energy, obtained through electric
vehicles and solar power.

Tesla began production of its first car model, the Roadster sports car,
in 2008. This was followed by the Model S sedan in 2012, the Model X SUV in
2015, the Model 3 sedan in 2017, the Model Y crossover in 2020, and the
Tesla Semi truck in 2022. The company plans production of the Cybertruck
light-duty pickup truck in 2023.[8]
The Model 3 is the all-time bestselling plug-in electric car worldwide,
and in June 2021 became the first electric car to sell 1 million units
globally.[9]
Tesla's 2022 deliveries were around 1.31 million vehicles, a 40% increase
over the previous year,[10][11] and cumulative sales totaled 4 million cars
as of April 2023.[12] In October 2021,
Tesla's market capitalization temporarily reached $1 trillion, the sixth
 company to do so in U.S. history.

Tesla has been the subject of lawsuits, government scrutiny, and
journalistic criticism, stemming from allegations of whistleblower
retaliation, worker rights violations, product defects,
and Musk's many controversial statements.
"""
print(llm_chain.run(text1))

 • Tesla, Inc. is an American multinational automotive and clean energy company
            • Headquartered in Austin, Texas, Tesla designs and manufactures electric vehicles, stationary battery energy storage devices, solar panels, and solar roof tiles
            • Tesla is one of the world's most valuable companies and led the battery electric vehicle market in 2022 with 18% share
            • Subsidiary Tesla Energy develops and installs photovoltaic systems in the US, and is a major supplier of battery energy storage systems
            • Elon Musk became CEO in 2008 and the company's mission is to expedite the move to sustainable transport and energy through electric vehicles and solar power
            • Tesla has produced several car models, including the Roadster sports car, Model S sedan, Model X SUV, Model 3 sedan, Model Y crossover, and Tesla Semi truck
            • The Model 3 is the all-time bestselling plug-in electric car worldwide, and Tesla's 2022 deliveries were ar

In [8]:
template = """
      Detect named entities in following text delimited by triple backquotes.
      Return your response in json format with spans of named entities with
      fields
      "named entity","type","span".
       Return all entities
       ```{text}```
       json format file:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(text1))

 [
            {
            "named_entity": "Tesla, Inc.",
            "type": "company",
            "span": "100-110"
            },
            {
            "named_entity": "Tesla Motors",
            "type": "company",
            "span": "100-110"
            },
            {
            "named_entity": "Nikola Tesla",
            "type": "person",
            "span": "120-130"
            },
            {
            "named_entity": "Elon Musk",
            "type": "person",
            "span": "140-150"
            },
            {
            "named_entity": "Martin Eberhard",
            "type": "person",
            "span": "160-170"
            },
            {
            "named_entity": "Marc Tarpenning",
            "type": "person",
            "span": "160-170"
            }
            ]
      
   

Note: The response is a JSON array of objects, where each object represents a named entity in the text, with fields for "named entity", "type", and "span". The "span" fie

In [24]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [25]:
!pip install langchain

In [26]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [27]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.

In our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds of the government offices for this purpose. The government should reach him. Since you have assigned this responsibility to me, about four crore families have got their ‘pucca’ houses. However, I have encountered cases where someone is left out of the government benefits. Therefore, I have decided to tour the country again, to listen to people’s experiences with government schemes, to understand whether they received the intended benefits, and to ensure that the programs are reaching everyone as planned without paying any bribes. We will get the real picture if we visit them again. Therefore, this ‘Viksit Bharat Sankalp Yatra’ is, in a way, my own examination. I want to hear from you and the people across the country whether what I envisioned and the work I have been doing aligns with reality and whether it has reached those for whom it was meant.

It is crucial to check whether the work that was supposed to happen has indeed taken place. I recently met some individuals who utilized the Ayushman card to get treatment for serious illnesses. One person met with a severe accident, and after using the card, he could afford the necessary operation, and now he is recovering well. When I asked him, he said: “How could I afford this treatment? Now that there is the Ayushman card, I mustered courage and underwent an operation. Now I am perfectly fine.”  Such stories are blessings to me.

The bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel satisfied that 50 or 100 people who were supposed to get the funds have got it. The funds meant for a thousand villages have been released. But their job satisfaction peaks when they hear that their work has directly impacted someone’s life positively. When they see the tangible results of their efforts, their enthusiasm multiplies. They feel satisfied. Therefore, ‘Viksit Bharat Sankalp Yatra’ has had a positive impact on government officers. It has made them more enthusiastic about their work, especially when they witness the tangible benefits reaching the people. Officers now feel satisfied with their work, saying, “I made a good plan, I created a file, and the intended beneficiaries received the benefits.” When they find that the money has reached a poor widow under the Jeevan Jyoti scheme and it was a great help to her during her crisis, they realise that they have done a good job. When a government officer listens to such stories, he feels very satisfied.

There are very few who understand the power and impact of the ‘Viksit Bharat Sankalp Yatra’. When I hear people connected to bureaucratic circles talking about it, expressing their satisfaction, it resonates with me. I’ve heard stories where someone suddenly received 2 lakh rupees after the death of her husband, and a sister mentioned how the arrival of gas in her home transformed her lives. The most significant aspect is when someone says that the line between rich and poor has vanished. While the slogan ‘Garibi Hatao’ (Remove Poverty) is one thing, but the real change happens when a person says, “As soon as the gas stove came to my house, the distinction between poverty and affluence disappeared.
"""

chat_messages=[
    SystemMessage(content='You are an expert assistant with expertize in summarizing speeches'),
    HumanMessage(content=f'Please provide a short and concise summary of the following speech:\n TEXT: {speech}')
]

In [30]:
llm(chat_messages)

ValueError: Argument `prompt` is expected to be a string. Instead found <class 'list'>. If you want to run the LLM on multiple prompts, use `generate` instead.

In [29]:
llm.get_num_tokens(speech)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

909

In [37]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [38]:
from PyPDF2 import PdfReader

In [40]:
pdfreader = PdfReader('/Get_Started_With_Smallpdf.pdf')

In [41]:
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [42]:
text

'Welcome to Smallpdf\nDigital Documents—All In One Place\nAccess Files Anytime, Anywhere Enhance Documents in One Click \nCollaborate With Others With the new Smallpdf experience, you can \nfreely upload, organize, and share digital \ndocuments. When you enable the ‘Storage’ \noption, we’ll also store all processed files here. \nYou can access files stored on Smallpdf from \nyour computer, phone, or tablet. We’ll also \nsync files from the Smallpdf Mobile App to our \nonline portalWhen you right-click on a file, we’ll present \nyou with an array of options to convert, \ncompress, or modify it. \nForget mundane administrative tasks. With \nSmallpdf, you can request e-signatures, send \nlarge files, or even enable the Smallpdf G Suite \nApp for your entire organization. Ready to take document management to the next level? \n'

In [43]:
from langchain.docstore.document import Document

In [44]:
docs = [Document(page_content=text)]
docs

[Document(page_content='Welcome to Smallpdf\nDigital Documents—All In One Place\nAccess Files Anytime, Anywhere Enhance Documents in One Click \nCollaborate With Others With the new Smallpdf experience, you can \nfreely upload, organize, and share digital \ndocuments. When you enable the ‘Storage’ \noption, we’ll also store all processed files here. \nYou can access files stored on Smallpdf from \nyour computer, phone, or tablet. We’ll also \nsync files from the Smallpdf Mobile App to our \nonline portalWhen you right-click on a file, we’ll present \nyou with an array of options to convert, \ncompress, or modify it. \nForget mundane administrative tasks. With \nSmallpdf, you can request e-signatures, send \nlarge files, or even enable the Smallpdf G Suite \nApp for your entire organization. Ready to take document management to the next level? \n')]

In [45]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [46]:
template = '''Write a concise and short summary of the following speech.
Speech: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [47]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [48]:
output_summary


'Summary: The speech introduces Smallpdf, a platform that allows users to upload, organize, share, and edit digital documents in one place. It also offers features such as file conversion, compression, and e-signatures. The platform is accessible from any device and offers a seamless experience for individuals and organizations.'

In [49]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [50]:
llm.get_num_tokens(text)

213

In [51]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [52]:
len(chunks)

1

In [53]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
summary = chain.run(chunks)

In [54]:
summary

'\nSmallpdf is a digital document platform that allows users to upload, organize, and share files from their computer, phone, or tablet. The platform offers various features such as file conversion, compression, and modification, as well as e-signature request, large file sending, and integration with G Suite.'

In [55]:

chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [56]:
final_combine_prompt='''
Provide a final summary of the entire speech with these important points.
Add a Generic Motivational Title,
Start the precise summary with an introduction and provide the
summary in number points for the speech.
Speech: `{text}`
'''
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [57]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [58]:
output

'\nFinal Summary:\n\n* Smallpdf is a platform that allows users to store, organize, and share digital documents in one place.\n* The platform offers various features such as uploading, processing, and sharing documents, as well as converting, compressing, or modifying them.\n* Users can access files from their computer, phone, or tablet.\n* The platform offers e-signature request, large file sending, and integration with G Suite for organizations.\n* The speaker invites the audience to try out the platform and take document management to the next level.\n\nGeneric Motivational Title: `Unlocking the Power of Document Management`\n\nSummary of the Speech in Number Points:\n\n1. Smallpdf is a platform that allows users to store, organize, and share digital documents in one place.\n2. The platform offers various features such as uploading, processing, and sharing documents, as well as converting, compressing, or modifying them.\n3. Users can access files from their computer, phone, or tabl

In [59]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Welcome to Smallpdf
Digital Documents—All In One Place
Access Files Anytime, Anywhere Enhance Documents in One Click 
Collaborate With Others With the new Smallpdf experience, you can 
freely upload, organize, and share digital 
documents. When you enable the ‘Storage’ 
option, we’ll also store all processed files here. 
You can access files stored on Smallpdf from 
your computer, phone, or tablet. We’ll also 
sync files from the Smallpdf Mobile App to our 
online portalWhen you right-click on a file, we’ll present 
you with an array of options to convert, 
compress, or modify it. 
Forget mundane administrative tasks. With 
Smallpdf, you can request e-signatures, send 
large files, or even enable the Smallpdf G Suite 
App for your entire organization. Ready to take document management to the next level?"


CONCISE SUMMARY:

> Finished c

In [60]:
output_summary

'\nSmallpdf is a platform that allows users to upload, organize, and share digital documents. It offers storage options, enabling users to access files from any device. The platform also provides options for converting, compressing, or modifying files. Additionally, users can request e-signatures, send large files, and even enable the Smallpdf G Suite App for their organization.'